In [ ]:
from tastytrade.common.logging import setup_logging
import logging
import asyncio
import pandas as pd
import polars as pl
from tastytrade.connections import Credentials

from tastytrade.config.enumerations import Channels
from tastytrade.connections.sockets import DXLinkManager

from tastytrade.analytics.visualizations.custom import plot_live_candlesticks
from tastytrade.analytics.visualizations.charts import DynamicChart, Study
from datetime import datetime, timedelta

from tastytrade.messaging.processors.influxdb import TelegrafHTTPEventProcessor

from tastytrade.analytics.indicators.momentum import hull

# Show all rows in pandas DataFrames
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

logging.getLogger().handlers.clear()

TEST = True
ENV = "Live"
DURATION = 15

EDT = 5

start_time = datetime(2020, 1, 1, 0, 00)
end_time = start_time + timedelta(hours=8)

setup_logging(
    level=logging.INFO,
    log_dir="../logs",
    filename_prefix=f"{'dev' if TEST else 'prod'}_tastytrade",
    console=True,
    file=True,
)

loop = asyncio.get_event_loop()
loop.set_debug(True)
logging.getLogger("asyncio").setLevel(logging.DEBUG)

## Test individual components

In [ ]:
# Set API credentials
credentials = Credentials(env=ENV)


credentials = Credentials(env="Live")
dxlink = DXLinkManager()

await dxlink.open(credentials=credentials)

In [ ]:
dxlink.router.add_processor(TelegrafHTTPEventProcessor())

In [ ]:
dxlink.router.handler[Channels.Candle].processors

In [5]:
symbol = "BTC/USD:CXTALP" 
# symbol = "NVDA"
symbol = "SPX"
# symbol = "SPY"
# symbol = "QQQ"

In [6]:
await dxlink.subscribe_to_candles(
    symbol=symbol,
    interval="1d",
    from_time=start_time,
)

In [7]:
await dxlink.subscribe_to_candles(
    symbol=symbol,
    interval="1h",
    from_time=start_time,
)

In [8]:
await dxlink.subscribe_to_candles(
    symbol=symbol,
    interval="30m",
    from_time=start_time,
)

In [9]:
await dxlink.subscribe_to_candles(
    symbol=symbol,
    interval="15m",
    from_time=start_time,
)

In [10]:
await dxlink.subscribe_to_candles(
    symbol=symbol,
    interval="5m",
    from_time=start_time,
)

In [11]:
await dxlink.subscribe_to_candles(
    symbol=symbol,
    interval="1m",
    from_time=start_time,
)

In [ ]:
await dxlink.close()

In [9]:
df = dxlink.router.handler[Channels.Candle].processors["feed"].pl.clone()
await dxlink.close()

from dataclasses import dataclass
import pytz


@dataclass
class Event:
    eventSymbol: str
    time: datetime
    open: float
    high: float
    low: float
    close: float


event = Event(
    eventSymbol="BTC/USD:CXTALP{=m}",
    time=datetime(2025, 2, 14, 00, 1),
    open=100,
    high=100,
    low=100,
    close=100,
)

df.sort("index")

In [8]:
# symbols = ["SPX"]
# symbols = ["SPX", "NVDA", "BTC/USD:CXTALP"]
symbols = ["BTC/USD:CXTALP"]
await dxlink.subscribe(symbols)

In [9]:
# await dxlink.unsubscribe(symbols)

In [ ]:
print(
    dxlink.router.handler[Channels.Candle]
    .processors["feed"]
    .df.groupby("eventSymbol")["time"]
    .max()
)

In [ ]:
print(
    dxlink.router.handler[Channels.Candle]
    .processors["feed"]
    .df.groupby("eventSymbol")["time"]
    .max()
)

In [ ]:
# Access candle data
columns = [
    "eventSymbol",
    "time",
    "open",
    "high",
    "low",
    "close",
    "tradeDate",
    "tradeTime",
    "prevOpen",
    "prevHigh",
    "prevLow",
    "prevClose",
    "prevDate",
    "prevTime",
]
dxlink.router.handler[Channels.Candle].processors["feed"].df[columns].tail(10)

In [13]:
df = dxlink.router.handler[Channels.Candle].processors["feed"].pl.clone()

# df = df.with_columns(
#     pl.col("time")
#     .dt.convert_time_zone("America/New_York")
#     .dt.strftime("%Y-%m-%d")
#     .alias("tradeDate"),
#     pl.col("time").dt.convert_time_zone("America/New_York").dt.strftime("%H:%M").alias("tradeTime"),
# )

In [ ]:
df

In [ ]:
(
    df.sort("time", descending=False)
    .filter(pl.col("eventSymbol") == event.eventSymbol)
    .filter(pl.col("time").lt(event.time))
    .tail(1)
)

In [13]:
def get_previous_candle(candle_df, event):

    if candle_df.is_empty():
        return {}

    candle = (
        candle_df.sort("time", descending=False)
        .filter(pl.col("eventSymbol") == event.eventSymbol)
        .filter(pl.col("time") < event.time)
        .tail(1)
    )

    return {} if candle.is_empty() else candle.to_dicts().pop()

In [ ]:
get_previous_candle(df, event)

In [35]:
for col in ["open", "high", "low", "close", "tradeDate", "tradeTime"]:
    df = df.with_columns(pl.col(col).shift(1).over("eventSymbol").alias(f"prev_{col}"))

In [ ]:
df.select(
    pl.col("eventSymbol"),
    pl.col("tradeDate"),
    pl.col("prev_tradeDate"),
    pl.col("tradeTime"),
    pl.col("prev_tradeTime"),
    pl.col("open"),
    pl.col("prev_open"),
)

In [ ]:
dxlink.router.handler[Channels.Trade].processors["feed"].df.tail(10)

In [ ]:
dxlink.router.handler[Channels.Quote].processors["feed"].df.tail(10)

In [ ]:
dxlink.router.handler[Channels.Greeks].processors["feed"].df

In [ ]:
dxlink.router.handler[Channels.Profile].processors["feed"].df

In [ ]:
dxlink.router.handler[Channels.Summary].processors["feed"].df

In [ ]:
candle_symbol_1m = "BTC/USD:CXTALP{=m}"
candle_symbol_5m = "BTC/USD:CXTALP{=5m}"

In [ ]:
# Example usage
study_params = {"hull": {"length": 20, "price_col": "close"}}  # HMA period  # Price column to use

# Create the live chart with HMA
task = plot_live_candlesticks(dxlink=dxlink, symbol=candle_symbol_1m)

In [ ]:
# Later when you want to stop
task.cancel()

In [ ]:
task = plot_live_candlesticks(dxlink, candle_symbol_5m)

In [ ]:
task.cancel()

In [ ]:
hull(dxlink, candle_symbol_1m).head(3)

In [19]:
hma_study = Study(
    name="HMA-20",
    compute_fn=hull,  # The hull function
    params={"length": 20},  # Parameters for hull function
    plot_params={
        "colors": {"Up": "#26A69A", "Down": "#EF5350"},  # Green for uptrend  # Red for downtrend
        "width": 2,
    },
    value_column="HMA",
    color_column="HMA_color",
)

In [ ]:
chart = DynamicChart(dxlink, candle_symbol_1m)
chart.add_study(hma_study)
chart.start()
# chart.stop()

In [ ]:
chart2 = DynamicChart(dxlink, candle_symbol_5m)
chart2.add_study(hma_study)
chart2.start()
# chart2.stop()

In [ ]:
await dxlink.close()

In [24]:
# TODOS

# Widen the plot
# Remove the scroller at the bottom
# move the legent and remove Price (that is obvious)

# Add MACD
# Add RSI
# Add Volume Profile (?? ... /ES, SPY, etc)
# Add velocity metric

# Do not plot study data points passed available data

# HULL - Align the first tick against to the subsequent tick

# ERROR if no study data found

In [7]:
from tastytrade.messaging.models.events import TradeEvent
from datetime import datetime

trade = TradeEvent(
    eventSymbol="NVIDIA",
    time=datetime(2025, 2, 9, 10, 0),
    price=100,
    size=1,
)

processor = TelegrafHTTPEventProcessor()
processor.process_event(trade)

In [ ]:
import influxdb_client
import os

write_client = influxdb_client.InfluxDBClient(
    url="http://influxdb:8086",
    token=os.environ["INFLUX_DB_TOKEN"],
    org=os.environ["INFLUX_DB_ORG"],
)

query_api = write_client.query_api()

query = """from(bucket: "tastytrade")
 |> range(start: -10d)
 |> filter(fn: (r) => r._measurement == "TradeEvent")"""

tables = query_api.query(query, org=os.environ["INFLUX_DB_ORG"])

for table in tables:
    print(table)
    for record in table.records:
        print(record)

In [ ]:
tables

In [ ]:
symbol = "SPX{=d}"
df.sort("time", descending=False).filter(pl.col("eventSymbol") == symbol).is_empty()

df.sort("time", descending=False).filter(pl.col("eventSymbol") == symbol)

In [ ]:
symbol = "SPX{=1d}"
df.sort("time", descending=False).filter(pl.col("eventSymbol") == symbol).is_empty()

In [ ]:
def get_last_candle(df: pl.DataFrame, symbol: str) -> pl.DataFrame:
    candle = df.sort("time", descending=False).filter(pl.col("eventSymbol") == symbol).tail(1)
    return None if candle.is_empty() else candle.to_dicts().pop()


get_last_candle(df, "SPX{=d}")

In [ ]:
from tastytrade.messaging.processors.default import CandleEventProcessor
from tastytrade.messaging.models.events import BasicCandleEvent

candle_processor = CandleEventProcessor()
candle_processor.process_event(
    BasicCandleEvent(
        eventSymbol="SPX", time=datetime(2025, 2, 14, 10, 0), open=100, high=100, low=100, close=100
    )
)
candle_processor.df